In [0]:
dbutils.widgets.text('p_file_date','2021-03-21')
v_file_date=dbutils.widgets.get('p_file_date')

In [0]:
print(v_file_date)


In [0]:
spark.sql("""
          CREATE TABLE IF NOT EXISTS f1_presentation.calculated_race_results
          (
              race_year INT,
              team_name STRING,
              driver_name STRING,
              driver_id INT,
              race_id INT,
              position INT,
              points INT,
              cal_points INT,
              created_date TIMESTAMP,
              updated_date TIMESTAMP
          )
          using delta
        """)


In [0]:


spark.sql(f"""
CREATE OR REPLACE TEMP VIEW race_result_view AS
SELECT
    races.race_year,
    constructors.name AS team_name,
    drivers.name AS driver_name,
    drivers.driver_id,
    races.race_id,
    results.position,
    results.points,
    11 - results.position AS cal_points
FROM f1_processed.results
JOIN f1_processed.drivers ON results.driver_id = drivers.driver_id
JOIN f1_processed.constructors ON results.constructor_id = constructors.constructor_id
JOIN f1_processed.races ON results.race_id = races.race_id
WHERE results.position <= 10
  AND results.file_date = '{v_file_date}'
""")



In [0]:
%sql
select COUNT(1) FROM race_result_view

In [0]:
%sql
MERGE INTO f1_presentation.calculated_race_results tgt
USING race_result_view v
ON tgt.race_id = v.race_id AND tgt.driver_id = v.driver_id
WHEN MATCHED THEN 
UPDATE SET 
    tgt.position = v.position,
    tgt.points = v.points,
    tgt.cal_points = v.cal_points,
    tgt.updated_date = current_timestamp
WHEN NOT MATCHED THEN
INSERT (
    race_year,
    team_name,
    driver_id,
    driver_name,
    race_id,
    position,
    points,
    cal_points,
    created_date
) VALUES (
    v.race_year,
    v.team_name,
    v.driver_id,
    v.driver_name,
    v.race_id,
    v.position,
    v.points,
    v.cal_points,
    current_timestamp
)

In [0]:
%sql
select * from f1_presentation.calculated_race_results